In [1]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import tabula

In [9]:
urls = []
r = requests.get('https://trreb.ca/index.php/market-news/rental-market-report/rental-market-report-archive')
soup = BeautifulSoup(r.text)
for x in soup.find_all('a'):
    url = x.get('href')
    if re.search('/market-stats/rental-reports/', url) is not None:
        urls.append(url)

In [10]:
urls

['/files/market-stats/rental-reports/rental_report_Q4-2021.pdf',
 '/files/market-stats/rental-reports/rental_report_Q3-2021.pdf',
 '/files/market-stats/rental-reports/rental_report_Q2-2021.pdf',
 '/files/market-stats/rental-reports/rental_report_Q1-2021.pdf',
 '/files/market-stats/rental-reports/rental_report_Q4-2020.pdf',
 '/files/market-stats/rental-reports/rental_report_Q3-2020.pdf',
 '/files/market-stats/rental-reports/rental_report_Q2-2020.pdf',
 '/files/market-stats/rental-reports/rental_report_Q1-2020.pdf',
 '/files/market-stats/rental-reports/rental_report_Q4-2019.pdf',
 '/files/market-stats/rental-reports/rental_report_Q3-2019.pdf',
 '/files/market-stats/rental-reports/rental_report_Q2-2019.pdf',
 '/files/market-stats/rental-reports/rental_report_Q1-2019.pdf',
 '/files/market-stats/rental-reports/rental_report_Q4-2018.pdf',
 '/files/market-stats/rental-reports/rental_report_Q3-2018.pdf',
 '/files/market-stats/rental-reports/rental_report_Q2-2018.pdf',
 '/files/market-stats/ren

In [30]:
[*range(1,5,1)]

[1, 2, 3, 4]

In [31]:
years = [*range(2012,2022,1)]
quarters = [*range(1,5,1)]

# for year in years:
#     for quarter in quarters:
#         response = requests.get(f"https://trreb.ca/files/market-stats/rental-reports/rental_report_Q{quarter}-{year}.pdf")
#         with open(f'../data/raw/rental/Q{quarter}-{year}.pdf', 'wb') as f:
#             f.write(response.content)

In [129]:
year = years[3]
quarter = quarters[3]

# print(year, quarter)


2015 4


Got stderr: Apr 02, 2022 11:20:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 02, 2022 11:20:04 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [143]:
dfs = []
for year in years:
    for quarter in quarters:
        print(year, quarter)
        file1 = f'../data/raw/rental/Q{quarter}-{year}.pdf'
        table = tabula.read_pdf(file1, pages = '3', stream=True, area = (76.725,31.482,566.775,764.082))[0]
        table.drop(table.index[0:(np.where(table.iloc[:,0] == 'Toronto West')[0][0]-1)], inplace=True)
        table.dropna(axis=1, how='all', inplace=True)
        table.set_index(table.columns[0], inplace=True)
        splits = [table[col].str.split(pat=' ', expand=True).add_prefix(col) for col in table.columns]
        table = pd.concat(splits, axis=1)
        table = table.reset_index(drop=False)
        col1 = ['Region'] + [x for x in ['All_Apartments', 'Bachelor', 'One_Bedroom', 'Two_Bedroom', 'Three_Bedroom'] for _ in (0, 1)]
        col2 = ["", "Total_Listed", "Total_Leased"] + ["Bachelor_Leased", "Avg_Lease_Rate"]*4
        table.columns = ['_'.join([x,y]).strip('_') for x, y in zip(col1,col2)]
        table = table.loc[table['Region'].apply(lambda x: re.search('\d+', x) is not None)]
        table['Region'] = table['Region'].apply(lambda x: re.search('\w\d{2}', x).group())
        table.iloc[:,1:] = table.iloc[:,1:].apply(lambda x_row: pd.Series([int(''.join(re.findall('\d+', x))) if re.findall('\d+', x) != [] else np.nan for x in x_row.values]), axis = 1).values
        table['Year'] = year
        table['Quarter'] = quarter
        table.replace(0, np.nan, inplace=True)
        table.reset_index(drop=True, inplace=True)
        dfs.append(table)

2012 1
2012 2
2012 3
2012 4
2013 1
2013 2
2013 3
2013 4
2014 1
2014 2
2014 3
2014 4
2015 1
2015 2
2015 3


Got stderr: Apr 02, 2022 11:30:46 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 02, 2022 11:30:46 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



2015 4


Got stderr: Apr 02, 2022 11:30:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 02, 2022 11:30:48 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



2016 1


...

2019 3


Got stderr: Apr 02, 2022 11:31:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 02, 2022 11:31:11 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



2019 4


Got stderr: Apr 02, 2022 11:31:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 02, 2022 11:31:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



2020 1
2020 2
2020 3
2020 4
2021 1
2021 2
2021 3
2021 4


In [144]:
pd.concat(dfs, axis=0)

,Region,All_Apartments_Total_Listed,All_Apartments_Total_Leased,Bachelor_Bachelor_Leased,Bachelor_Avg_Lease_Rate,One_Bedroom_Bachelor_Leased,One_Bedroom_Avg_Lease_Rate,Two_Bedroom_Bachelor_Leased,Two_Bedroom_Avg_Lease_Rate,Three_Bedroom_Bachelor_Leased,Three_Bedroom_Avg_Lease_Rate,Year,Quarter
0,W01,66.0,32.0,NaN,NaN,24.0,1539.0,8.0,2031.0,NaN,NaN,2012,1
1,W02,13.0,9.0,NaN,NaN,3.0,1633.0,6.0,1679.0,NaN,NaN,2012,1
2,W03,3.0,3.0,NaN,NaN,2.0,1050.0,1.0,1650.0,NaN,NaN,2012,1
3,W04,10.0,2.0,NaN,NaN,1.0,1150.0,1.0,1350.0,NaN,NaN,2012,1
4,W05,43.0,17.0,NaN,NaN,12.0,1354.0,5.0,1578.0,NaN,NaN,2012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,E07,125.0,99.0,NaN,NaN,41.0,1976.0,54.0,2408.0,4.0,2645.0,2021,4
31,E08,73.0,35.0,NaN,NaN,12.0,1824.0,16.0,2215.0,7.0,2543.0,2021,4
32,E09,226.0,188.0,5.0,1698.0,111.0,2080.0,68.0,2770.0,4.0,3275.0,2021,4
33,E10,32.0,22.0,NaN,NaN,14.0,1813.0,7.0,2399.0,1.0,2700.0,2021,4


In [124]:
table.copy(deep=True)

,Region,All_Apartments_Total_Listed,All_Apartments_Total_Leased,Bachelor_Bachelor_Leased,Bachelor_Avg_Lease_Rate,One_Bedroom_Bachelor_Leased,One_Bedroom_Avg_Lease_Rate,Two_Bedroom_Bachelor_Leased,Two_Bedroom_Avg_Lease_Rate,Three_Bedroom_Bachelor_Leased,Three_Bedroom_Avg_Lease_Rate,Year,Quarter
0,W01,195.0,102.0,NaN,NaN,70.0,1632.0,31.0,2110.0,1.0,2190.0,2015,2
1,W02,76.0,56.0,NaN,NaN,35.0,1516.0,21.0,2304.0,NaN,NaN,2015,2
2,W03,4.0,2.0,NaN,NaN,1.0,1400.0,NaN,NaN,1.0,1900.0,2015,2
3,W04,108.0,72.0,NaN,NaN,51.0,1305.0,18.0,1644.0,3.0,1750.0,2015,2
4,W05,100.0,39.0,2.0,1123.0,23.0,1413.0,12.0,1612.0,2.0,2000.0,2015,2
5,W06,850.0,482.0,2.0,1038.0,276.0,1448.0,202.0,2012.0,2.0,2925.0,2015,2
6,W07,13.0,9.0,NaN,NaN,9.0,1565.0,NaN,NaN,NaN,NaN,2015,2
7,W08,331.0,182.0,1.0,1210.0,113.0,1454.0,66.0,1909.0,2.0,2363.0,2015,2
8,W09,12.0,8.0,NaN,NaN,1.0,955.0,7.0,1699.0,NaN,NaN,2015,2
9,W10,43.0,31.0,NaN,NaN,11.0,1237.0,20.0,1481.0,NaN,NaN,2015,2


In [26]:
["Total_Listed", "Total_Leased", "Bachelor_Leased", "Avg_Lease_Rate", "Bachelor_Leased", "Avg_Lease_Rate", "Bachelor_Leased", "Avg_Lease_Rate", "Bachelor_Leased", "Avg_Lease_Rate"]

,Region,All_Apartments_Total_Listed,All_Apartments_Total_Leased,Bachelor_Leased,Bachelor_Avg_Lease_Rate,One_Bedroom_Leased,One_Bedroom_Avg_Lease_Rate,Two_Bedroom_Leased,Two_Bedroom_Avg_Lease_Rate,Three_Bedroom_Leased,Three_Bedroom_Avg_Lease_Rate,Year,Quarter
4,W01,117,58,0,NaN,36,2104,20,3096,2,4125,2021,4
5,W02,99,60,2,1400,45,2055,13,2851,0,NaN,2021,4
6,W03,10,5,0,NaN,3,1833,1,2200,1,2850,2021,4
7,W04,105,64,0,NaN,38,1890,23,2271,3,2808,2021,4
8,W05,106,71,2,1600,45,1924,22,2254,2,2550,2021,4
9,W06,536,319,2,1695,201,2147,112,2930,4,5875,2021,4
10,W07,38,26,0,NaN,20,2038,6,2766,0,NaN,2021,4
11,W08,408,244,2,1663,140,1943,98,2549,4,3094,2021,4
12,W09,24,15,0,NaN,2,2400,9,2139,4,2863,2021,4
13,W10,85,46,0,NaN,19,1804,24,2223,3,2483,2021,4


In [39]:
table

,Region,All_Apartments_Total_Listed,Bachelor_Total_Leased,One_Bedroom_Leased,Two_Bedroom_Avg_Lease_Rate,Three_Bedroom_Leased,All_Apartments_Avg_Lease_Rate,Bachelor_Leased,One_Bedroom_Avg_Lease_Rate,Two_Bedroom_Leased,Three_Bedroom_Avg_Lease_Rate
4,W01,199,86,1,1850,50,2016,31,2997,4,4038
5,W02,184,74,0,0,49,1905,24,2685,1,2250
6,W03,24,10,0,0,7,1928,3,2400,0,0
7,W04,227,100,2,1430,57,1724,37,2130,4,2525
8,W05,97,35,2,1398,23,1831,10,2175,0,0
9,W06,815,351,6,1403,193,1993,149,2579,3,4667
10,W07,27,11,0,0,5,1610,6,2727,0,0
11,W08,551,216,0,0,134,1850,78,2294,4,2650
12,W09,30,15,0,0,1,1750,12,2333,2,2325
13,W10,92,34,0,0,11,1850,20,2075,3,2300


In [6]:
table['Region'] = table.apply(lambda x: re.search('\w\d{2}', x['Region']).group()).values

KeyError: 'Region'

In [37]:
table